In [1]:
import pandas as pd
from dask_ml.linear_model import LinearRegression
from dask_ml.compose import ColumnTransformer
from dask_ml.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from tqdm import tqdm
from sklearn.utils.validation import check_is_fitted
from sklearn.ensemble import RandomForestRegressor
from torch.utils.data import DataLoader,Dataset
from xgboost import XGBRegressor
import dask.dataframe as df
import numpy as np
import torch.optim as optim
import torch
import torch.nn as nn
import warnings

### Loading data

In [3]:
#clients id, treatment and their target
Train_clients = pd.read_csv('/home/luchian/all_data/uni_data/upliftX5_data/train.csv')
print(Train_clients.info())
Train_clients.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140027 entries, 0 to 140026
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   client_id      140027 non-null  object
 1   treatment_flg  140027 non-null  int64 
 2   target         140027 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.2+ MB
None


,client_id,treatment_flg,target
0,ad6561e2d8,1,1
1,7c1ccbf93f,1,1
2,b58fadcab6,1,1
3,e99e6fabb9,0,0
4,27fb6f8520,1,1


In [4]:
Test_clients = pd.read_csv('/home/luchian/all_data/uni_data/upliftX5_data/test.csv')
print(Test_clients.info())
Test_clients.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60012 entries, 0 to 60011
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  60012 non-null  object
dtypes: object(1)
memory usage: 469.0+ KB
None


,client_id
0,a9a604ed6e
1,ebd7360016
2,908cd9b8e8
3,dceb8ce861
4,f4f0ac6b06


In [5]:
all_clients_info = pd.read_csv('/home/luchian/all_data/uni_data/upliftX5_data/clients2.csv', usecols = ['client_id','first_issue_date','first_redeem_date','age','gender'])
print(all_clients_info.info())
all_clients_info.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200039 entries, 0 to 200038
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   client_id          200039 non-null  object
 1   first_issue_date   200039 non-null  object
 2   first_redeem_date  182493 non-null  object
 3   age                200039 non-null  int64 
 4   gender             200039 non-null  object
dtypes: int64(1), object(4)
memory usage: 7.6+ MB
None


,client_id,first_issue_date,first_redeem_date,age,gender
0,000012768d,2017-08-05 15:40:48,2018-01-04 19:30:07,45,U
1,000036f903,2017-04-10 13:54:23,2017-04-23 12:37:56,72,F
2,00010925a5,2018-07-24 16:21:29,2018-09-14 16:12:49,83,U
3,0001f552b0,2017-06-30 19:20:38,2018-08-28 12:59:45,33,F
4,00020e7b18,2017-11-27 11:41:45,2018-01-10 17:50:05,73,U


In [6]:
all_products = pd.read_csv('/home/luchian/all_data/uni_data/upliftX5_data/products.csv')
print(all_products.info())
all_products.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43038 entries, 0 to 43037
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   product_id        43038 non-null  object 
 1   level_1           43035 non-null  object 
 2   level_2           43035 non-null  object 
 3   level_3           43035 non-null  object 
 4   level_4           43035 non-null  object 
 5   segment_id        41466 non-null  float64
 6   brand_id          37838 non-null  object 
 7   vendor_id         43004 non-null  object 
 8   netto             43035 non-null  float64
 9   is_own_trademark  43038 non-null  int64  
 10  is_alcohol        43038 non-null  int64  
dtypes: float64(2), int64(2), object(7)
memory usage: 3.6+ MB
None


,product_id,level_1,level_2,level_3,level_4,segment_id,brand_id,vendor_id,netto,is_own_trademark,is_alcohol
0,0003020d3c,c3d3a8e8c6,c2a3ea8d5e,b7cda0ec0c,6376f2a852,123.0,394a54a7c1,9eaff48661,0.400,0,0
1,0003870676,e344ab2e71,52f13dac0c,d3cfe81323,6dc544533f,105.0,acd3dd483f,10486c3cf0,0.680,0,0
2,0003ceaf69,c3d3a8e8c6,f2333c90fb,419bc5b424,f6148afbc0,271.0,f597581079,764e660dda,0.500,0,0
3,000701e093,ec62ce61e3,4202626fcb,88a515c084,48cf3d488f,172.0,54a90fe769,03c2d70bad,0.112,0,0
4,0007149564,e344ab2e71,52f13dac0c,d3cfe81323,6dc544533f,105.0,63417fe1f3,f329130198,0.600,0,0


In [14]:
train_purchases = pd.read_csv('/home/luchian/all_data/uni_data/upliftX5_data/train_purch.csv') #takes 17 minutes to load

In [7]:
pd.set_option("display.max_info_rows", 16_000_000) 
print(train_purchases.info(verbose = True))
train_purchases.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15998952 entries, 0 to 15998951
Data columns (total 13 columns):
 #   Column                   Non-Null Count     Dtype  
---  ------                   --------------     -----  
 0   client_id                15998952 non-null  object 
 1   transaction_id           15998952 non-null  object 
 2   transaction_datetime     15998952 non-null  object 
 3   regular_points_received  15998952 non-null  float64
 4   express_points_received  15998952 non-null  float64
 5   regular_points_spent     15998952 non-null  float64
 6   express_points_spent     15998952 non-null  float64
 7   purchase_sum             15998952 non-null  float64
 8   store_id                 15998952 non-null  object 
 9   product_id               15998952 non-null  object 
 10  product_quantity         15998952 non-null  float64
 11  trn_sum_from_iss         15998952 non-null  float64
 12  trn_sum_from_red         1055864 non-null   float64
dtypes: float64(8), object(5)


,client_id,transaction_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_id,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,9a80204f78,2.0,80.0,NaN
1,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,da89ebd374,1.0,65.0,NaN
2,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,0a95e1151d,1.0,24.0,NaN
3,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,4055b15e4a,2.0,50.0,NaN
4,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,a685f1916b,1.0,22.0,NaN


In [14]:
train_purchases.drop('transaction_id',axis = 1,inplace=True)
train_purchases.head(5)

,client_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_id,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,000012768d,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,9a80204f78,2.0,80.0,NaN
1,000012768d,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,da89ebd374,1.0,65.0,NaN
2,000012768d,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,0a95e1151d,1.0,24.0,NaN
3,000012768d,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,4055b15e4a,2.0,50.0,NaN
4,000012768d,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,a685f1916b,1.0,22.0,NaN


### Data Processing

In [9]:
#it took about 6 minutes execute (need to save this)
# train_clients_mask = all_clients_info['client_id'].map(lambda x: Train_clients['client_id'].isin([x]).any())
# train_clients_mask

In [10]:
#saving it ... 
# train_clients_info = all_clients_info[train_clients_mask]
# train_clients_info.head(5)

In [12]:
#loading info of training calients
train_clients_info = pd.read_csv('train_clients_info.csv')
train_clients_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140027 entries, 0 to 140026
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   client_id          140027 non-null  object
 1   first_issue_date   140027 non-null  object
 2   first_redeem_date  127673 non-null  object
 3   age                140027 non-null  int64 
 4   gender             140027 non-null  object
dtypes: int64(1), object(4)
memory usage: 5.3+ MB


In [7]:
class DateTimeSplitter(BaseEstimator, TransformerMixin):
    """
    splits one of the datetime dtype column into separate columns
    """
    
    def __init__(self,
                 input_year = 1991,
                 components=['year','month','day','hour','minute','second'],
                 errors='coerce',
                 utc=False):
        self.input_year = input_year
        self.components = components
        self.errors = errors
        self.utc = utc
        
    def fit(self, X, y=None):
        """Store input feature names"""
        if not isinstance(X, np.ndarray):
            raise ValueError("Input must be numpy array")
            
        # Store valid components
        valid_components = ['year','month','day','hour','minute','second','dayofweek','quarter']
        self.components_ = [c for c in self.components if c in valid_components]
        return self
        
    def transform(self, X):
        """Split datetime column into components"""
        check_is_fitted(self)
        # Convert to datetime
        datetime_series = pd.to_datetime(pd.Series(X.reshape(-1)))
        
        # Extract components
        component_data = {}
        for comp in self.components_:
            if comp == 'year':
                component_data[comp] = datetime_series.dt.year
            elif comp == 'month':
                component_data[comp] = datetime_series.dt.month
            elif comp == 'day':
                component_data[comp] = datetime_series.dt.day
            elif comp == 'hour':
                component_data[comp] = datetime_series.dt.hour
            elif comp == 'minute':
                component_data[comp] = datetime_series.dt.minute
            elif comp == 'second':
                component_data[comp] = datetime_series.dt.second
            elif comp == 'dayofweek':
                component_data[comp] = datetime_series.dt.dayofweek
            elif comp == 'quarter':
                component_data[comp] = datetime_series.dt.quarter
        
        ret_value = pd.DataFrame(component_data).to_numpy()
        ret_value[ret_value[:,0] == self.input_year] = np.zeros(len(self.components_),dtype = np.int8)
        return ret_value
    
    def get_feature_names_out(self):
        return self.components_

In [8]:
def process_dates(the_frame,cols,copy = True):
    """Applies simple transformaions to datetime columns"""
    input_date = '1921-11-11 20:20:20'
    #if too large we do not copy
    if copy == True:
        result_frame = the_frame.copy()
    else:
        result_frame = the_frame

    list_of_frames = []
    #the main pipeline
    for col in cols:
        fill_nan = ColumnTransformer([
            ('with_dummy', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value = input_date), [col]),
        ])
        date_trans = Pipeline([
            ('initial_trans',fill_nan),
            ('splitted_date',DateTimeSplitter(input_year=1921))
        ])
        #for each column apply the transformation
        #and concatenate afterwards
        result = date_trans.fit_transform(the_frame)
        col_names = [col+'_'+ date_part for date_part in date_trans[-1].get_feature_names_out()]
        list_of_frames.append(pd.DataFrame(result,columns=col_names))
        result_frame.drop(col,inplace = True,axis = 1)

    #concatenate the cols and drop the old ones
    for one_frame in list_of_frames:
        result_frame = pd.concat([result_frame,one_frame],axis = 1)

    #drop the seconds cols
    for col in cols:
        result_frame.drop(col+'_'+'second',inplace = True,axis = 1)
    return result_frame


In [9]:
def encode_impute_feature_cols(dataframe, enc_cols,imp_cols, copy = True):
    #if the data is too large we just do not copy it
    if copy == True:
        new_frame = dataframe.copy()
    else:
        new_frame = dataframe

    for one_col in enc_cols:
        ord_enc = OrdinalEncoder()
        new_frame[[one_col]] = ord_enc.fit_transform(new_frame[[one_col]])
    for one_col in imp_cols:
        freq_imp = SimpleImputer(strategy='most_frequent')
        new_frame[[one_col]] = freq_imp.fit_transform(new_frame[[one_col]])
    return new_frame

In [10]:
def make_mutual_features(list_of_frames,keycol):
    return pd.merge(*list_of_frames, on = keycol, how = 'inner',copy = False)

In [16]:
#need to process the train_purchases
#need to process the products_data

In [17]:
train_purchases

,client_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_id,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,000012768d,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,9a80204f78,2.0,80.0,NaN
1,000012768d,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,da89ebd374,1.0,65.0,NaN
2,000012768d,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,0a95e1151d,1.0,24.0,NaN
3,000012768d,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,4055b15e4a,2.0,50.0,NaN
4,000012768d,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,a685f1916b,1.0,22.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
15998947,fffeca6d22,2019-03-09 07:47:43,0.4,0.0,0.0,0.0,99.0,a0613e36b5,de437430e2,1.0,19.0,NaN
15998948,fffeca6d22,2019-03-09 07:47:43,0.4,0.0,0.0,0.0,99.0,a0613e36b5,ad8fee4200,1.0,45.0,NaN
15998949,fffeca6d22,2019-03-15 14:46:39,0.8,0.0,0.0,0.0,165.0,a0613e36b5,ad8fee4200,2.0,94.0,NaN
15998950,fffeca6d22,2019-03-15 14:46:39,0.8,0.0,0.0,0.0,165.0,a0613e36b5,230dbb0c16,1.0,50.0,NaN


In [15]:
proc_train_purch = encode_impute_feature_cols(process_dates(train_purchases, cols = ['transaction_datetime'], copy  = False),
                                              enc_cols = ['store_id'], imp_cols = ['trn_sum_from_red'],copy = False)

In [19]:
proc_train_purch.head(5)

,client_id,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_id,product_quantity,trn_sum_from_iss,trn_sum_from_red,transaction_datetime_year,transaction_datetime_month,transaction_datetime_day,transaction_datetime_hour,transaction_datetime_minute
0,000012768d,10.0,0.0,0.0,0.0,1007.0,4556.0,9a80204f78,2.0,80.0,5.0,2018,12,1,7,12
1,000012768d,10.0,0.0,0.0,0.0,1007.0,4556.0,da89ebd374,1.0,65.0,5.0,2018,12,1,7,12
2,000012768d,10.0,0.0,0.0,0.0,1007.0,4556.0,0a95e1151d,1.0,24.0,5.0,2018,12,1,7,12
3,000012768d,10.0,0.0,0.0,0.0,1007.0,4556.0,4055b15e4a,2.0,50.0,5.0,2018,12,1,7,12
4,000012768d,10.0,0.0,0.0,0.0,1007.0,4556.0,a685f1916b,1.0,22.0,5.0,2018,12,1,7,12


In [20]:
proc_train_purch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15998952 entries, 0 to 15998951
Data columns (total 16 columns):
 #   Column                       Non-Null Count     Dtype  
---  ------                       --------------     -----  
 0   client_id                    15998952 non-null  object 
 1   regular_points_received      15998952 non-null  float64
 2   express_points_received      15998952 non-null  float64
 3   regular_points_spent         15998952 non-null  float64
 4   express_points_spent         15998952 non-null  float64
 5   purchase_sum                 15998952 non-null  float64
 6   store_id                     15998952 non-null  float64
 7   product_id                   15998952 non-null  object 
 8   product_quantity             15998952 non-null  float64
 9   trn_sum_from_iss             15998952 non-null  float64
 10  trn_sum_from_red             15998952 non-null  float64
 11  transaction_datetime_year    15998952 non-null  int32  
 12  transaction_datetime_month

In [17]:
proc_all_products = encode_impute_feature_cols(all_products, enc_cols = ['level_1', 'level_2', 'level_3', 'level_4', 'segment_id', 'brand_id', 'vendor_id'], imp_cols = ['level_1', 'level_2', 'level_3', 'level_4', 'segment_id', 'brand_id', 'vendor_id', 'netto'])
proc_all_products.head(5)

,product_id,level_1,level_2,level_3,level_4,segment_id,brand_id,vendor_id,netto,is_own_trademark,is_alcohol
0,0003020d3c,0.0,30.0,153.0,296.0,44.0,925.0,2053.0,0.400,0,0
1,0003870676,1.0,12.0,173.0,325.0,35.0,2855.0,221.0,0.680,0,0
2,0003ceaf69,0.0,37.0,53.0,752.0,102.0,4103.0,1504.0,0.500,0,0
3,000701e093,2.0,10.0,104.0,210.0,64.0,1362.0,65.0,0.112,0,0
4,0007149564,1.0,12.0,173.0,325.0,35.0,1605.0,3023.0,0.600,0,0


In [18]:
proc_all_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43038 entries, 0 to 43037
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   product_id        43038 non-null  object 
 1   level_1           43038 non-null  float64
 2   level_2           43038 non-null  float64
 3   level_3           43038 non-null  float64
 4   level_4           43038 non-null  float64
 5   segment_id        43038 non-null  float64
 6   brand_id          43038 non-null  float64
 7   vendor_id         43038 non-null  float64
 8   netto             43038 non-null  float64
 9   is_own_trademark  43038 non-null  int64  
 10  is_alcohol        43038 non-null  int64  
dtypes: float64(8), int64(2), object(1)
memory usage: 3.6+ MB


In [13]:
proc_train_clients_info = process_dates(train_clients_info,['first_issue_date','first_redeem_date'],copy = True)
proc_train_clients_info.head(5)

,client_id,age,gender,first_issue_date_year,first_issue_date_month,first_issue_date_day,first_issue_date_hour,first_issue_date_minute,first_redeem_date_year,first_redeem_date_month,first_redeem_date_day,first_redeem_date_hour,first_redeem_date_minute
0,000012768d,45,U,2017,8,5,15,40,2018,1,4,19,30
1,000036f903,72,F,2017,4,10,13,54,2017,4,23,12,37
2,0001f552b0,33,F,2017,6,30,19,20,2018,8,28,12,59
3,00020e7b18,73,U,2017,11,27,11,41,2018,1,10,17,50
4,000220a0a7,115,M,2017,12,9,17,26,2018,10,4,14,59


In [14]:
proc_train_clients_info = encode_impute_feature_cols(proc_train_clients_info,enc_cols=['gender'],imp_cols=[])
proc_train_clients_info.head(5)

,client_id,age,gender,first_issue_date_year,first_issue_date_month,first_issue_date_day,first_issue_date_hour,first_issue_date_minute,first_redeem_date_year,first_redeem_date_month,first_redeem_date_day,first_redeem_date_hour,first_redeem_date_minute
0,000012768d,45,2.0,2017,8,5,15,40,2018,1,4,19,30
1,000036f903,72,0.0,2017,4,10,13,54,2017,4,23,12,37
2,0001f552b0,33,0.0,2017,6,30,19,20,2018,8,28,12,59
3,00020e7b18,73,2.0,2017,11,27,11,41,2018,1,10,17,50
4,000220a0a7,115,1.0,2017,12,9,17,26,2018,10,4,14,59


In [25]:
proc_train_clients_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140027 entries, 0 to 140026
Data columns (total 13 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   client_id                 140027 non-null  object 
 1   age                       140027 non-null  int64  
 2   gender                    140027 non-null  float64
 3   first_issue_date_year     140027 non-null  int32  
 4   first_issue_date_month    140027 non-null  int32  
 5   first_issue_date_day      140027 non-null  int32  
 6   first_issue_date_hour     140027 non-null  int32  
 7   first_issue_date_minute   140027 non-null  int32  
 8   first_redeem_date_year    140027 non-null  int32  
 9   first_redeem_date_month   140027 non-null  int32  
 10  first_redeem_date_day     140027 non-null  int32  
 11  first_redeem_date_hour    140027 non-null  int32  
 12  first_redeem_date_minute  140027 non-null  int32  
dtypes: float64(1), int32(10), int64(1), object(1

In [16]:
#assamble everything in one feature X
X_v1 = make_mutual_features([proc_train_clients_info,proc_train_purch],'client_id')
X_v1.head(5)

,client_id,age,gender,first_issue_date_year,first_issue_date_month,first_issue_date_day,first_issue_date_hour,first_issue_date_minute,first_redeem_date_year,first_redeem_date_month,...,store_id,product_id,product_quantity,trn_sum_from_iss,trn_sum_from_red,transaction_datetime_year,transaction_datetime_month,transaction_datetime_day,transaction_datetime_hour,transaction_datetime_minute
0,000012768d,45,U,2017,8,5,15,40,2018,1,...,4556.0,9a80204f78,2.0,80.0,5.0,2018,12,1,7,12
1,000012768d,45,U,2017,8,5,15,40,2018,1,...,4556.0,da89ebd374,1.0,65.0,5.0,2018,12,1,7,12
2,000012768d,45,U,2017,8,5,15,40,2018,1,...,4556.0,0a95e1151d,1.0,24.0,5.0,2018,12,1,7,12
3,000012768d,45,U,2017,8,5,15,40,2018,1,...,4556.0,4055b15e4a,2.0,50.0,5.0,2018,12,1,7,12
4,000012768d,45,U,2017,8,5,15,40,2018,1,...,4556.0,a685f1916b,1.0,22.0,5.0,2018,12,1,7,12


In [27]:
# X_v1.to_csv('~/all_data/uni_data/X_v1.csv') to much in RAM, so saving is needed

In [15]:
X_v1 = df.read_csv('~/all_data/uni_data/X_v1.csv',index_col = False).drop('Unnamed: 0',axis = 1)

In [19]:
df_proc_all_products = df.from_pandas(proc_all_products)
df_proc_all_products

,product_id,level_1,level_2,level_3,level_4,segment_id,brand_id,vendor_id,netto,is_own_trademark,is_alcohol
npartitions=1,,,,,,,,,,,
0,string,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64
43037,...,...,...,...,...,...,...,...,...,...,...


In [20]:
df_train_clients = df.from_pandas(Train_clients)
df_train_clients.head(5)

,client_id,treatment_flg,target
0,ad6561e2d8,1,1
1,7c1ccbf93f,1,1
2,b58fadcab6,1,1
3,e99e6fabb9,0,0
4,27fb6f8520,1,1


In [21]:
df_proc_all_products.head(5)

,product_id,level_1,level_2,level_3,level_4,segment_id,brand_id,vendor_id,netto,is_own_trademark,is_alcohol
0,0003020d3c,0.0,30.0,153.0,296.0,44.0,925.0,2053.0,0.400,0,0
1,0003870676,1.0,12.0,173.0,325.0,35.0,2855.0,221.0,0.680,0,0
2,0003ceaf69,0.0,37.0,53.0,752.0,102.0,4103.0,1504.0,0.500,0,0
3,000701e093,2.0,10.0,104.0,210.0,64.0,1362.0,65.0,0.112,0,0
4,0007149564,1.0,12.0,173.0,325.0,35.0,1605.0,3023.0,0.600,0,0


In [22]:
X_v1.head(5)

,client_id,age,gender,first_issue_date_year,first_issue_date_month,first_issue_date_day,first_issue_date_hour,first_issue_date_minute,first_redeem_date_year,first_redeem_date_month,...,store_id,product_id,product_quantity,trn_sum_from_iss,trn_sum_from_red,transaction_datetime_year,transaction_datetime_month,transaction_datetime_day,transaction_datetime_hour,transaction_datetime_minute
0,000012768d,45,2.0,2017,8,5,15,40,2018,1,...,4556.0,9a80204f78,2.0,80.0,5.0,2018,12,1,7,12
1,000012768d,45,2.0,2017,8,5,15,40,2018,1,...,4556.0,da89ebd374,1.0,65.0,5.0,2018,12,1,7,12
2,000012768d,45,2.0,2017,8,5,15,40,2018,1,...,4556.0,0a95e1151d,1.0,24.0,5.0,2018,12,1,7,12
3,000012768d,45,2.0,2017,8,5,15,40,2018,1,...,4556.0,4055b15e4a,2.0,50.0,5.0,2018,12,1,7,12
4,000012768d,45,2.0,2017,8,5,15,40,2018,1,...,4556.0,a685f1916b,1.0,22.0,5.0,2018,12,1,7,12


In [23]:
X_v2 = df.merge(left = df_proc_all_products,right = X_v1, on = 'product_id', how = 'inner').drop('product_id',axis = 1)
X_v2

,level_1,level_2,level_3,level_4,segment_id,brand_id,vendor_id,netto,is_own_trademark,is_alcohol,client_id,age,gender,first_issue_date_year,first_issue_date_month,first_issue_date_day,first_issue_date_hour,first_issue_date_minute,first_redeem_date_year,first_redeem_date_month,first_redeem_date_day,first_redeem_date_hour,first_redeem_date_minute,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_quantity,trn_sum_from_iss,trn_sum_from_red,transaction_datetime_year,transaction_datetime_month,transaction_datetime_day,transaction_datetime_hour,transaction_datetime_minute
npartitions=31,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,string,int64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [24]:
X_v2 = df.merge(left = df_proc_all_products,right = X_v1, on = 'product_id', how = 'inner').drop('product_id',axis = 1)

In [25]:
X_v3 = df.merge(left = df_train_clients,right = X_v2, on = 'client_id', how = 'inner').drop('client_id',axis = 1)
X_v3

,treatment_flg,target,level_1,level_2,level_3,level_4,segment_id,brand_id,vendor_id,netto,is_own_trademark,is_alcohol,age,gender,first_issue_date_year,first_issue_date_month,first_issue_date_day,first_issue_date_hour,first_issue_date_minute,first_redeem_date_year,first_redeem_date_month,first_redeem_date_day,first_redeem_date_hour,first_redeem_date_minute,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_quantity,trn_sum_from_iss,trn_sum_from_red,transaction_datetime_year,transaction_datetime_month,transaction_datetime_day,transaction_datetime_hour,transaction_datetime_minute
npartitions=31,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [26]:
X_v3.to_csv('/home/luchian/all_data/uni_data/X-*.csv')

['/home/luchian/all_data/uni_data/X-00.csv',
 '/home/luchian/all_data/uni_data/X-01.csv',
 '/home/luchian/all_data/uni_data/X-02.csv',
 '/home/luchian/all_data/uni_data/X-03.csv',
 '/home/luchian/all_data/uni_data/X-04.csv',
 '/home/luchian/all_data/uni_data/X-05.csv',
 '/home/luchian/all_data/uni_data/X-06.csv',
 '/home/luchian/all_data/uni_data/X-07.csv',
 '/home/luchian/all_data/uni_data/X-08.csv',
 '/home/luchian/all_data/uni_data/X-09.csv',
 '/home/luchian/all_data/uni_data/X-10.csv',
 '/home/luchian/all_data/uni_data/X-11.csv',
 '/home/luchian/all_data/uni_data/X-12.csv',
 '/home/luchian/all_data/uni_data/X-13.csv',
 '/home/luchian/all_data/uni_data/X-14.csv',
 '/home/luchian/all_data/uni_data/X-15.csv',
 '/home/luchian/all_data/uni_data/X-16.csv',
 '/home/luchian/all_data/uni_data/X-17.csv',
 '/home/luchian/all_data/uni_data/X-18.csv',
 '/home/luchian/all_data/uni_data/X-19.csv',
 '/home/luchian/all_data/uni_data/X-20.csv',
 '/home/luchian/all_data/uni_data/X-21.csv',
 '/home/lu

In [2]:
X_features = df.read_csv('/home/luchian/all_data/uni_data/X-*.csv').drop('Unnamed: 0',axis = 1).repartition(npartitions = 5)
X_features

,treatment_flg,target,level_1,level_2,level_3,level_4,segment_id,brand_id,vendor_id,netto,is_own_trademark,is_alcohol,age,gender,first_issue_date_year,first_issue_date_month,first_issue_date_day,first_issue_date_hour,first_issue_date_minute,first_redeem_date_year,first_redeem_date_month,first_redeem_date_day,first_redeem_date_hour,first_redeem_date_minute,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_quantity,trn_sum_from_iss,trn_sum_from_red,transaction_datetime_year,transaction_datetime_month,transaction_datetime_day,transaction_datetime_hour,transaction_datetime_minute
npartitions=5,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [3]:
class VanillaNN(nn.Module):
    """Vanilla neural network model for T-learner"""
    def __init__(self, input_dim, hidden_dims=[64, 32], output_dim=1):
        super(VanillaNN, self).__init__()
        layers = []
        prev_dim = input_dim
        
        # Create hidden layers
        for dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, dim))
            layers.append(nn.ReLU())
            prev_dim = dim
        
        # Output layer
        layers.append(nn.Linear(prev_dim, output_dim))
        
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.model(x)

In [4]:
vanT = VanillaNN(37,[64,32,64],1).to(device = 'cuda')
vanC = VanillaNN(37,[64,32,64],1).to(device = 'cuda')

In [5]:
def train_model(model, dask_frame_data, epochs=1, lr=0.001,dev = 'cuda'):
    """
    Train a single model for T-learner
    Args:
        model (nn.Module): Model to train
        epochs (int): Number of training epochs
        lr (float): Learning rate
    Returns:
        Trained model
    """
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    model.train()
    for epoch in range(epochs):
        for _ in tqdm(range(600)):
            feature_batch = dask_frame_data.sample(frac = 10e-07).compute()
            y = feature_batch[['target']]
            X_batch = feature_batch.drop('target',axis = 1)
            #to numpy tensors
            y = y.to_numpy(dtype = np.float32)
            X_batch = X_batch.to_numpy(dtype = np.float32)
            #to PyTorch tensors
            y = torch.from_numpy(y).to(device = dev)
            X_batch = torch.from_numpy(X_batch).to(device=dev)

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y)
            loss.backward()
            # try:
            #     loss.backward()
            # except:
            #     return X_batch,outputs,y
            optimizer.step()
            if True:
                print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.6f}")
    
    return model

In [ ]:
res_treatment = train_model(vanT,X_features[X_features['treatment_flg'] == 1])

  0%|          | 1/600 [00:11<1:59:28, 11.97s/it]

Epoch 1/1, Loss: 662.397339


  0%|          | 2/600 [00:23<1:56:37, 11.70s/it]

Epoch 1/1, Loss: 47.638760


  0%|          | 3/600 [00:34<1:55:24, 11.60s/it]

...

Epoch 1/1, Loss: 25.624472


  2%|▏         | 11/600 [02:07<1:53:33, 11.57s/it]

Epoch 1/1, Loss: 26.167501


  2%|▏         | 12/600 [02:19<1:54:25, 11.68s/it]

Epoch 1/1, Loss: 51.547333


In [ ]:
res_control = train_model(vanC,X_features[X_features['treatment_flg'] == 0])

(tensor([[1.0000e+00, 0.0000e+00, 2.7000e+01, 1.8500e+02, 3.2000e+02, 2.4000e+01,
          2.6870e+03, 2.9560e+03, 2.5000e-01, 0.0000e+00, 0.0000e+00, 2.6000e+01,
          2.0000e+00, 2.0170e+03, 4.0000e+00, 2.4000e+01, 1.6000e+01, 4.9000e+01,
          2.0180e+03, 3.0000e+00, 2.2000e+01, 1.6000e+01, 4.0000e+01, 2.3000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 4.7015e+02, 1.9870e+03, 1.0000e+00,
          3.0000e+01, 5.0000e+00, 2.0190e+03, 1.0000e+00, 1.2000e+01, 9.0000e+00,
          1.1000e+01],
         [1.0000e+00, 1.0000e+00, 3.6000e+01, 1.6000e+02, 4.7000e+02, 9.9000e+01,
          2.1420e+03, 1.3770e+03, 1.0000e+00, 0.0000e+00, 0.0000e+00, 3.4000e+01,
          2.0000e+00, 2.0170e+03, 6.0000e+00, 2.5000e+01, 1.7000e+01, 3.1000e+01,
          2.0170e+03, 7.0000e+00, 9.0000e+00, 1.7000e+01, 1.6000e+01, 1.7000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 3.4260e+02, 7.7240e+03, 1.0000e+00,
          3.9000e+01, 5.0000e+00, 2.0190e+03, 3.0000e+00, 3.0000e+00, 9.000

In [3]:
class DaskDataset(Dataset):
    """
    Memory-efficient PyTorch Dataset for Dask DataFrames.
    Loads only necessary batches on-demand using Dask partitions.
    """
    def __init__(self, dask_df, feature_columns, target_column):
        """
        Args:
            dask_df (dd.DataFrame): Dask DataFrame containing the dataset
            feature_columns (list): List of feature column names
            target_column (str): Name of the target column
        """
        self.dask_df = dask_df
        self.feature_columns = feature_columns
        self.target_column = target_column
        self.length = len(dask_df)
        
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        if not hasattr(self, '_current_partition') or not self._in_current_partition(idx):
            self._load_partition_for_index(idx)
        
        local_idx = idx - self._partition_start
        row = self._current_partition.iloc[local_idx]
        
        features = torch.tensor(row[self.feature_columns].values.astype(np.float32))
        target = torch.tensor(row[self.target_column]).float()
        
        return features, target
    
    def _in_current_partition(self, idx):
        return self._partition_start <= idx < self._partition_end
    
    def _load_partition_for_index(self, idx):
        """Load the Dask partition containing the requested index"""
        # Find which partition contains the index
        partition_index = 0
        cumulative = 0
        for i, part in enumerate(self.dask_df.partitions):
            part_length = len(part)
            if cumulative <= idx < cumulative + part_length:
                partition_index = i
                self._partition_start = cumulative
                self._partition_end = cumulative + part_length
                break
            cumulative += part_length
        
        # Compute and cache the partition
        self._current_partition = self.dask_df.partitions[partition_index].compute()

In [ ]:
device = 'cuda'
class SLearner(nn.Module):
    """S-learner: Single model with treatment as feature"""
    def __init__(self, input_dim, hidden_dims=[64, 32]):
        super(SLearner, self).__init__()
        # Input includes both features and treatment indicator
        self.base_model = VanillaNN(input_dim + 1, hidden_dims)
    
    def forward(self, features, treatment):
        # Concatenate features with treatment indicator
        model_input = torch.cat([features, treatment.unsqueeze(1)], dim=1)
        return self.base_model(model_input)
    
    def predict_cate(self, features):
        """Predict CATE: μ(features, 1) - μ(features, 0)"""
        self.eval()
        with torch.no_grad():
            treated = self(features, torch.ones(len(features), 1))
            control = self(features, torch.zeros(len(features), 1))
        return treated - control


def train_s_learner(dataloader, input_dim, epochs=10, lr=0.001):
    """Train S-learner on full dataset"""
    model = SLearner(input_dim).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0.0
        for features, treatment, outcome in dataloader:
            optimizer.zero_grad()
            preds = model(features, treatment)
            loss = criterion(preds.squeeze(), outcome)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(dataloader):.6f}")
    
    return model


# Modified Dataset for S-learner (includes treatment indicator)
class SLearnerDataset(DaskDataset):
    def __getitem__(self, idx):
        if not hasattr(self, '_current_partition') or not self._in_current_partition(idx):
            self._load_partition_for_index(idx)
        
        local_idx = idx - self._partition_start
        row = self._current_partition.iloc[local_idx]
        
        features = torch.tensor(row[self.feature_columns].values.astype(np.float32))
        treatment = torch.tensor(row['treatment']).float()
        outcome = torch.tensor(row[self.target_column]).float()
        
        return features, treatment, outcome

In [6]:
def train_x_learner(full_dataloader, control_dataloader, treated_dataloader, input_dim, epochs=10):
    """X-learner implementation with 4 stages"""
    # Stage 1: Train initial models (T-learner)
    control_model, treated_model = create_t_learner(
        control_dataloader, treated_dataloader, input_dim
    )
    
    # Stage 2: Impute treatment effects
    control_imputed, treated_imputed = [], []
    
    # Process control group
    control_model.eval()
    for features, outcome in control_dataloader:
        with torch.no_grad():
            imputed_effect = treated_model(features) - outcome.unsqueeze(1)
            control_imputed.append(torch.cat([features, imputed_effect], dim=1))
    
    # Process treated group
    treated_model.eval()
    for features, outcome in treated_dataloader:
        with torch.no_grad():
            imputed_effect = outcome.unsqueeze(1) - control_model(features)
            treated_imputed.append(torch.cat([features, imputed_effect], dim=1))
    
    # Stage 3: Train tau models
    tau_control_model = VanillaNN(input_dim).to(device)
    tau_treated_model = VanillaNN(input_dim).to(device)
    
    # Create DataLoaders for imputed effects
    control_imputed_ds = torch.utils.data.TensorDataset(torch.cat(control_imputed)[:, :-1], torch.cat(control_imputed)[:, -1])
    treated_imputed_ds = torch.utils.data.TensorDataset(torch.cat(treated_imputed)[:, :-1], torch.cat(treated_imputed)[:, -1])
    
    control_imputed_loader = DataLoader(control_imputed_ds, batch_size=32, shuffle=True)
    treated_imputed_loader = DataLoader(treated_imputed_ds, batch_size=32, shuffle=True)
    
    # Train tau models
    tau_control_model = train_model(tau_control_model, control_imputed_loader, epochs=epochs)
    tau_treated_model = train_model(tau_treated_model, treated_imputed_loader, epochs=epochs)
    
    # Stage 4: Train propensity model (optional)
    # In practice we'd use a separate model, but for simplicity:
    prop_model = VanillaNN(input_dim, output_dim=1).to(device)
    prop_criterion = nn.BCEWithLogitsLoss()
    
    # Create propensity dataset
    prop_features, prop_labels = [], []
    for features, treatment, _ in full_dataloader:
        prop_features.append(features)
        prop_labels.append(treatment)
    
    prop_ds = torch.utils.data.TensorDataset(torch.cat(prop_features), torch.cat(prop_labels))
    prop_loader = DataLoader(prop_ds, batch_size=32, shuffle=True)
    
    # Train propensity model
    optimizer = optim.Adam(prop_model.parameters())
    for epoch in range(epochs//2):  # Fewer epochs
        for features, treatment in prop_loader:
            optimizer.zero_grad()
            logits = prop_model(features)
            loss = prop_criterion(logits.squeeze(), treatment)
            loss.backward()
            optimizer.step()
    
    return {
        'control_model': control_model,
        'treated_model': treated_model,
        'tau_control': tau_control_model,
        'tau_treated': tau_treated_model,
        'propensity_model': prop_model
    }


def predict_x_cate(x_models, features):
    """Predict CATE with X-learner"""
    with torch.no_grad():
        # Get propensity scores
        logits = x_models['propensity_model'](features)
        g_x = torch.sigmoid(logits)
        
        # Get tau predictions
        tau0 = x_models['tau_control'](features)
        tau1 = x_models['tau_treated'](features)
        
        # Weighted combination
        return g_x * tau0 + (1 - g_x) * tau1

In [7]:
class RLearner:
    """R-learner using Neyman-orthogonal scores"""
    def __init__(self, input_dim, hidden_dims=[64, 32]):
        self.outcome_model = VanillaNN(input_dim, hidden_dims)
        self.propensity_model = VanillaNN(input_dim, hidden_dims, output_dim=1)
        self.tau_model = VanillaNN(input_dim, hidden_dims)
    
    def train(self, dataloader, epochs=10, lr=0.001):
        # Stage 1: Train outcome and propensity models
        self._train_aux_models(dataloader, epochs, lr)
        
        # Stage 2: Train tau model with R-loss
        self._train_tau_model(dataloader, epochs, lr)
    
    def _train_aux_models(self, dataloader, epochs, lr):
        """Train outcome and propensity models"""
        # Train outcome model
        outcome_optim = optim.Adam(self.outcome_model.parameters(), lr=lr)
        outcome_criterion = nn.MSELoss()
        
        # Train propensity model
        prop_optim = optim.Adam(self.propensity_model.parameters(), lr=lr)
        prop_criterion = nn.BCEWithLogitsLoss()
        
        for epoch in range(epochs):
            outcome_loss, prop_loss = 0.0, 0.0
            for features, treatment, outcome in dataloader:
                # Train outcome model
                outcome_optim.zero_grad()
                out_preds = self.outcome_model(features)
                out_loss = outcome_criterion(out_preds.squeeze(), outcome)
                out_loss.backward()
                outcome_optim.step()
                outcome_loss += out_loss.item()
                
                # Train propensity model
                prop_optim.zero_grad()
                prop_logits = self.propensity_model(features)
                p_loss = prop_criterion(prop_logits.squeeze(), treatment)
                p_loss.backward()
                prop_optim.step()
                prop_loss += p_loss.item()
            
            print(f"Stage1 Epoch {epoch+1}/{epochs}, "
                  f"Outcome Loss: {outcome_loss/len(dataloader):.4f}, "
                  f"Propensity Loss: {prop_loss/len(dataloader):.4f}")
    
    def _train_tau_model(self, dataloader, epochs, lr):
        """Train tau model using R-learner loss"""
        tau_optim = optim.Adam(self.tau_model.parameters(), lr=lr)
        
        for epoch in range(epochs):
            total_loss = 0.0
            for features, treatment, outcome in dataloader:
                # Compute residuals
                with torch.no_grad():
                    y_res = outcome - self.outcome_model(features).squeeze()
                    p_score = torch.sigmoid(self.propensity_model(features).squeeze())
                    t_res = treatment - p_score
                
                # Compute R-loss
                tau_optim.zero_grad()
                tau_pred = self.tau_model(features).squeeze()
                loss = torch.mean((y_res - tau_pred * t_res) ** 2)
                
                loss.backward()
                tau_optim.step()
                total_loss += loss.item()
            
            print(f"R-Learner Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataloader):.6f}")
    
    def predict_cate(self, features):
        self.tau_model.eval()
        with torch.no_grad():
            return self.tau_model(features)


# Dataset for R-learner (same as S-learner)
class RLearnerDataset(SLearnerDataset):
    pass  # Identical structure to S-learner